# 작사가 만들기

### 데이터 불러오기

In [1]:
# 모듈 불러오기

import glob
import os
import re
import numpy as np
import tensorflow as tf


In [2]:
# 데이터 읽어오기
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:5])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth']


---

### 데이터 정제하기

In [3]:
# 문장 정제 함수 만들기
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백 지우기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백 넣기
    sentence = re.sub(r'[" "]+', " ", sentence) #  여러개의 공백을 하나의 공백으로 바꾸기
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸기
    sentence = sentence.strip() # 위의 것들 바꾸면서 다시 생긴 양쪽 공백 지우기
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에 <start>, 끝에 <end> 추가해주기
    return sentence

print("완료")


완료


In [4]:
# 정제된 문장 모으기
corpus = []

for sentence in raw_corpus:
    preprocessed_sentence = preprocess_sentence(sentence) # 문장 정제 먼저 하기
    if len(sentence) == 0: continue # 빈 문장이면 제외하기
    if sentence[-1] == ":": continue # ":"으로 끝나면 제외하기
    if len(preprocessed_sentence.split()) > 15: continue # 토큰 길이 15 넘어가면 제외하기
    corpus.append(preprocessed_sentence) # 문장 모아주기
    
# 정제된 결과 확인
print(corpus[:10])
len(corpus)

['<start> now i ve heard there was a secret chord <end>', '<start> that david played , and it pleased the lord <end>', '<start> but you don t really care for music , do you ? <end>', '<start> it goes like this <end>', '<start> the fourth , the fifth <end>', '<start> the minor fall , the major lift <end>', '<start> the baffled king composing hallelujah hallelujah <end>', '<start> hallelujah <end>', '<start> hallelujah <end>', '<start> hallelujah your faith was strong but you needed proof <end>']


156013

In [5]:
# Tokenizer 함수 만들기

def tokenize(corpus):
    # 12,000 단어 이상의 단어를 기억할 수 있는 tokenizer 만들기
    # 12,000 단어에 포함되지 못한 단어는 '<unk>'로 바꾸기
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000,
                                                     filters=' ',
                                                     oov_token="<unk>")
    # corpus 이용해서 tokenizer 내부의 단어장 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰주기
    # 시퀀스가 짧다면 문장 뒤에 패딩 붙여서 길이 맞춰주기 (padding='post')
    # (만약 문장 앞에 패딩 붙여서 길이 맞추고 싶으면 padding='pre'를 사용하기)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

# tokenize() 함수로 데이터를 Tensor로 변환해주기
tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f7bcefa88e0>


In [6]:
print(tensor[:3]) # 모두 정수로 이루어져 있음, 이 숫자들은 tokenizer에 구축된 단어 사전의 인덱스

[[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0
     0]
 [   2   15 2967  871    5    8   11 5739    6  374    3    0    0    0
     0]
 [   2   33    7   40   16  164  288   28  333    5   48    7   46    3
     0]]


In [7]:
tensor.shape # (정제된 문장들의 총 개수 : 156013개, 토큰의 최대 개수 : 15개)

(156013, 15)

In [8]:
# 단어사전이 어떻게 구축되어 있는지 확인해보기
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [9]:
# 생성된 텐서를 소스와 타겟으로 분리하기

# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


---

### 훈련, 평가 데이터셋 분리

In [10]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=42)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


---

### 훈련 데이터셋 생성

In [11]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 생성

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### TextGenerator 모델 만들기

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
# 한 배치만 불러온 데이터를 모델에 넣어보기
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-9.55585419e-05, -1.56625771e-04, -1.56642636e-04, ...,
          1.24469907e-05, -9.50043159e-06,  9.23849220e-05],
        [-7.85886914e-06, -2.17994413e-04, -4.67605918e-04, ...,
         -3.73620423e-05,  8.62866873e-05, -2.46967567e-04],
        [ 1.10928675e-04, -1.67456776e-04, -8.38014123e-04, ...,
          4.75334637e-05, -1.74995330e-05, -4.29671461e-04],
        ...,
        [ 4.08146996e-04,  1.53296941e-03,  1.15087285e-04, ...,
          1.91686529e-04,  9.33270436e-04, -2.75532046e-04],
        [ 4.33736684e-04,  2.04587914e-03,  2.80478504e-04, ...,
          3.89598776e-04,  8.05802352e-04, -5.92254873e-05],
        [ 4.24223166e-04,  2.51314719e-03,  3.74554191e-04, ...,
          5.39924949e-04,  6.25654473e-04,  1.15092625e-04]],

       [[-9.55585419e-05, -1.56625771e-04, -1.56642636e-04, ...,
          1.24469907e-05, -9.50043159e-06,  9.23849220e-05],
        [-1.30632223e-04, -5.90043128e-05, -3

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


 ---

### 모델 학습하기

In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
487/487 [==============================] - 78s 150ms/step - loss: 3.4941
Epoch 2/10
487/487 [==============================] - 77s 158ms/step - loss: 3.0142
Epoch 3/10
487/487 [==============================] - 79s 163ms/step - loss: 2.8520
Epoch 4/10
487/487 [==============================] - 81s 165ms/step - loss: 2.7303
Epoch 5/10
487/487 [==============================] - 82s 167ms/step - loss: 2.6260
Epoch 6/10
487/487 [==============================] - 82s 167ms/step - loss: 2.5323
Epoch 7/10
487/487 [==============================] - 82s 168ms/step - loss: 2.4461
Epoch 8/10
487/487 [==============================] - 82s 168ms/step - loss: 2.3664
Epoch 9/10
487/487 [==============================] - 82s 168ms/step - loss: 2.2917
Epoch 10/10
487/487 [==============================] - 82s 168ms/step - loss: 2.2217


### 평가 데이터셋 생성 및 모델 평가

In [16]:
BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 256
steps_per_epoch = len(enc_val) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 생성

test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
test_dataset = test_dataset.shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
test_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [17]:
# 모델 평가하기

val_loss = model.evaluate(test_dataset)

121/121 [==============================] - 9s 67ms/step - loss: 2.5268


---

### 작사가 만들기

In [18]:
# generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 함.

def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만들기
    while True:
        # 1. 입력받은 문장의 텐서를 입력
        predict = model(test_tensor) 
        # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아내기
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3. 2에서 예측된 word index를 문장 뒤에 붙이기
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마침
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

print("완료")

완료


---

### 문장 생성해보기

In [19]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you , liberian girl <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> i")

'<start> i m gonna be a little selfish <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> you")

'<start> you re the only one that s a star <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> long")

'<start> long as i got a few ass niggas <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> i can")

'<start> i can t help it <end> '

In [24]:
generate_text(model, tokenizer, init_sentence="<start> i miss")

'<start> i miss you , i m bad <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> it")

'<start> it s not a statement i m not <end> '

In [26]:
generate_text(model, tokenizer, init_sentence="<start> good")

'<start> good bye , i m a <unk> <end> '

In [27]:
generate_text(model, tokenizer, init_sentence="<start> have")

'<start> have you ever seen death singing <end> '

---

### 하이퍼파라미터 바꿔서 다시 해보기

In [28]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [29]:
# 모델 다시 학습하기

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
487/487 [==============================] - 244s 462ms/step - loss: 3.3631
Epoch 2/10
487/487 [==============================] - 227s 466ms/step - loss: 2.7992
Epoch 3/10
487/487 [==============================] - 227s 467ms/step - loss: 2.5182
Epoch 4/10
487/487 [==============================] - 227s 466ms/step - loss: 2.2432
Epoch 5/10
487/487 [==============================] - 227s 467ms/step - loss: 1.9801
Epoch 6/10
487/487 [==============================] - 227s 467ms/step - loss: 1.7353
Epoch 7/10
487/487 [==============================] - 227s 466ms/step - loss: 1.5219
Epoch 8/10
487/487 [==============================] - 227s 467ms/step - loss: 1.3434
Epoch 9/10
487/487 [==============================] - 227s 467ms/step - loss: 1.2025
Epoch 10/10
487/487 [==============================] - 227s 466ms/step - loss: 1.1009


In [30]:
# 모델 평가하기

val_loss = model.evaluate(test_dataset)

121/121 [==============================] - 21s 170ms/step - loss: 2.1834


In [32]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you , liberian girl , <end> '

In [33]:
generate_text(model, tokenizer, init_sentence="<start> i")

'<start> i m gonna be the mane event <end> '

In [34]:
generate_text(model, tokenizer, init_sentence="<start> you")

'<start> you know i m never gonna leave you <end> '

In [35]:
generate_text(model, tokenizer, init_sentence="<start> long")

'<start> long as i got stuff from most of em <end> '

In [37]:
generate_text(model, tokenizer, init_sentence="<start> i miss")

'<start> i miss you , i miss you <end> '

In [38]:
generate_text(model, tokenizer, init_sentence="<start> it")

'<start> it s a beautiful kind of pain <end> '

In [39]:
generate_text(model, tokenizer, init_sentence="<start> good")

'<start> good day sunshine . can t buy me love , love <end> '

In [40]:
generate_text(model, tokenizer, init_sentence="<start> have")

'<start> have you seen death singing <end> '

---

### 회고

val_loss가 처음에 2.5268이 나와서, 이것을 줄이기 위해서 하이퍼파라미터를 바꿔보았다.
- embedding_size = 256 -> 512
- hidden_size = 1024 -> 2048

하이퍼파라미터를 바꾸고 나서 확인해보니 val_loss가 2.1834로 줄었다.


조금 의아한건 하이퍼파라미터를 바꾸기 전의 노랫말들이 조금 더 자연스러운듯한 느낌이 든다. 물론 어법이 맞지 않고 완성이 안된 문장들도 있긴하다. 문장도 조금 짧은 느낌이 들긴 하다.

하이퍼파라미터를 바꾸고 난 뒤의 노랫말들은 조금 더 길어지긴 했는데, 단어 쓰임새가 틀린 경우도 있고, 약간 노랫말로 쓰기엔 어색한 느낌의 문장도 나온다.


확실히 자연어 처리는 조금 어렵다. 데이터 정제하는 것을 이해하는데도 조금 시간이 걸리고, RNN과 관련된 모델들도 아직 익숙하지 않아서 더 발전시키기가 어려운 것 같다.
나중에 공부를 좀더 해서 모델도 바꿔보고 하이퍼파라미터도 바꿔서 더 좋은 결과를 낼 수 있도록 해봐야겠다.

